![Logo de AA1](logo_AA1_texto_small.png) 
# Sesión 3 - Pandas II 
En esta práctica vamos a continuar viendo algunas funcionales interesantes de la librería `pandas` para el tratamiento y preparación de los datos mediante el uso de `DataFrames`.

**IMPORTANTE**: las funciones que vamos a ver en esta práctica pueden utilizarse de diversas maneras y con diferentes parámetros, lo que hará que su comportamiento varíe. Esta práctica no pretende explicar todoas las funcionalidades de la librería `pandas` (se necesitarían muchas horas para ello) sino que pretende únicamente ser una introducción a algunas de las características más importantes que tiene esta librería de cara a su uso en el tratamiento de conjuntos de datos. Información más detallada podréis encontrar en <https://pandas.pydata.org/docs/index.html>

Para ello vamos a trabajar con el conjunto de datos **Irish_certificate.xlsx**. Lo primero que haremos será cargar los datos como ya vimos en la sesión anterior:

In [1]:
# se importa la librería pandas nombrándola pd
import pandas as pd

# cargamos los datos de una hoja Excel
df = pd.read_excel('Irish_certificate.xlsx', sheet_name='Data', header=0)

display(df)

,Sex,DVRT,Educational_level,Prestige_score,Type_school,Leaving_certificate
0,male,113,Junior_cycle_incomplete-secondary_school,28.0,secondary,not_taken
1,male,101,Primary_terminal_leaver,28.0,primary_terminal_leaver,not_taken
2,male,110,Senior_cycle_terminal_leaver-secondary_school,69.0,secondary,taken
3,male,121,Junior_cycle_terminal_leaver-secondary_school,57.0,secondary,not_taken
4,male,82,Junior_cycle_terminal_leaver-vocational_school,18.0,vocational,not_taken
...,...,...,...,...,...,...
495,male,137,3rd_level_complete,62.0,secondary,taken
496,male,136,3rd_level_complete,18.0,secondary,taken
497,male,132,3rd_level_complete,37.0,secondary,taken
498,female,135,3rd_level_complete,62.0,secondary,taken


## 3.1 Agrupando ejemplos en función de valores
Vamos a ver una instrucción muy interesante que se llama `groupby()`.

A veces nos puede resultar interesante agrupar todos los ejemplos que comparten una misma característica para realizar algún cálculo sobre los mismos de forma agregada.

Por ejemplo, podría interesarnos agrupar a las personas por sexo para conocer la media del 'Prestige_score' en cada uno de los grupos. Se haría de la siguiente manera:

In [2]:
print('\n############## "Prestige_score" medio por sexo ##############')
display(df.groupby('Sex')['Prestige_score'].mean())


############## "Prestige_score" medio por sexo ##############


Sex
female    37.096639
male      40.788136
Name: Prestige_score, dtype: float64

Lo que ha hecho la instrucción es, agrupar los ejemplos por sexo, separar la columna 'Prestige_score' de cada grupo y calcular su media. En lugar de la media podríamos haber usado cualquier función que realice un calculo sobre los valores contenidos en el grupo. Podríamos usa, por ejemplo, `max()`, `min()`, `std()`, `count()`, `sum()`,...

Si lo que nos interesa el la media de todas las columnas de tipo numérico, entonces basta con no indicar una columna sobre la que hacer el cálculo. Si no le indicamos una columna, entonces realizará el cálculo sobre todas las columnas que pueda:

In [3]:
print('\n############## Cálculo de la media en las columnas por sexo ##############')
display(df.groupby('Sex').mean())


############## Cálculo de la media en las columnas por sexo ##############


,DVRT,Prestige_score
Sex,,
female,98.956,37.096639
male,101.348,40.788136


En este caso reliza el cálculo sobre las dos columnas numéricas que tiene el conjunto.

Si queremos también podemos realizar agrupaciones atendiendo a dos variables y realizar cálculos sobre los grupos resultantes:

In [4]:
print('\n############## Agrupando por sexo y tipo de escuela ##############')
display(df.groupby(['Sex', 'Type_school']).mean())


############## Agrupando por sexo y tipo de escuela ##############


DVRT  Prestige_score
Sex    Type_school                                        
female primary_terminal_leaver   83.882353       25.857143
       secondary                101.707182       40.200000
       vocational                94.307692       29.224490
male   primary_terminal_leaver   80.700000       31.368421
       secondary                108.395833       44.750000
       vocational                94.348837       36.345679

En este caso hemos agrupado por sexo y tipo de escuela y han resultado 6 grupos. El resultado de la instrucción genera un `DataFrame`que podemos almacenar en otra variable para acceder a los datos que nos interesen:

In [5]:
res = df.groupby(['Sex', 'Type_school']).mean()
display(type(res))

pandas.core.frame.DataFrame

Pero en `res`, ¿cuáles son los índices?. Ahora ya no nos aparecen numerados los ejemplos de 0 en adelante y eso puede despistarnos.

La propiedad `axes` almacena en un array los índices de los ejes. En este caso hay 2 ejes y podemos acceder a los índices de cada uno de ellos:

In [6]:
print('\n############## Consultamos los índices de todos los ejes ##############')
display(res.axes)

print('\n############## Consultamos los índices del eje 0 (filas) ##############')
display(res.axes[0])

print('\n############## Consultamos los índices del eje 1 (columnas) ##############')
display(res.axes[1])


############## Consultamos los índices de todos los ejes ##############


[MultiIndex([('female', 'primary_terminal_leaver'),
             ('female',               'secondary'),
             ('female',              'vocational'),
             (  'male', 'primary_terminal_leaver'),
             (  'male',               'secondary'),
             (  'male',              'vocational')],
            names=['Sex', 'Type_school']),
 Index(['DVRT', 'Prestige_score'], dtype='object')]


############## Consultamos los índices del eje 0 (filas) ##############


MultiIndex([('female', 'primary_terminal_leaver'),
            ('female',               'secondary'),
            ('female',              'vocational'),
            (  'male', 'primary_terminal_leaver'),
            (  'male',               'secondary'),
            (  'male',              'vocational')],
           names=['Sex', 'Type_school'])


############## Consultamos los índices del eje 1 (columnas) ##############


Index(['DVRT', 'Prestige_score'], dtype='object')

Vemos que son índices no numéricos, se utilizan etiquetas.

En el caso de las filas vemos que tiene un índice compuesto y en el de las columnas un índice normal. Vamos a ver ahora cómo podríamos acceder a los elementos de este `DataFrame` mediante el uso de `loc[]` e `iloc[]`.

In [7]:
print('\n############## Accedemos a una fila ##############')
display(res.loc[('female', 'vocational')])

print('\n############## Accedemos a un dato ##############')
display(res.loc[('male', 'secondary'), 'Prestige_score'])


############## Accedemos a una fila ##############


DVRT              94.307692
Prestige_score    29.224490
Name: (female, vocational), dtype: float64


############## Accedemos a un dato ##############


44.75

Para acceder con `loc[]` utilizamos las etiquetas como índices. En el caso de las filas es un índice compuesto, así que debemos utilizar una tupla, `('female', 'vocational')`, para indicar la fila a la que queremos acceder.

Si además le especificamos la columna entonces accederemos a un dato en concreto.

Ya habíamos comentado en la práctica anterior que los `DataFrames` siempre mantienen unos índices numéricos (aunque sea implícitamente). Así, podríamos utilizar `iloc[]` junto con esos índices numéricos para acceder a los mismos datos que en el ejemplo anterior:

In [8]:
print('\n############## Accedemos a la misma fila ##############')
display(res.iloc[2])

print('\n############## Accedemos al mismo dato ##############')
display(res.iloc[4, 1])


############## Accedemos a la misma fila ##############


DVRT              94.307692
Prestige_score    29.224490
Name: (female, vocational), dtype: float64


############## Accedemos al mismo dato ##############


44.75

Si queremos aplicar funciones diferentes dependiendo de la columna, entonces debemos utilizar `agg()` en combinación con `groupby()`:

In [9]:
display(df.groupby('Leaving_certificate').agg({'Prestige_score':['mean','std'],'DVRT':'min'}))

Prestige_score            DVRT
                              mean        std  min
Leaving_certificate                               
not_taken                34.570342  13.738347   65
taken                    44.374408  15.508741   70

## 3.2 Aplicando una función a todos los elementos de una columna

A veces puede resultar necesario aplicar una función a todos los elementos de una columna. En estos casos `pandas` cuenta con el método `apply()`, al cual podemos indicarle la función que queremos aplicar:

In [10]:
print('\n############## Definimos la función ##############')
def mayusculas(x):
    return x.upper()

print('\n############## Aplicamos la función ##############')
display(df['Sex'].apply(mayusculas))


############## Definimos la función ##############

############## Aplicamos la función ##############


0        MALE
1        MALE
2        MALE
3        MALE
4        MALE
        ...  
495      MALE
496      MALE
497      MALE
498    FEMALE
499    FEMALE
Name: Sex, Length: 500, dtype: object

In [11]:
print('\n############## Aplicamos la función ##############')
display(df['Sex'].apply(lambda x: x.upper()))


############## Aplicamos la función ##############


0        MALE
1        MALE
2        MALE
3        MALE
4        MALE
        ...  
495      MALE
496      MALE
497      MALE
498    FEMALE
499    FEMALE
Name: Sex, Length: 500, dtype: object

La instrucción anterior no modifica el `DataFrame` sino que se limita a generar el resultado. Si queremos que el cambio quede reflejado en `df` debemos asignar el resultado a la columna correspondiente:

In [17]:
print('\n############## Aplicamos la función ##############')
df['Sex'] = df['Sex'].apply(mayusculas)
display(df)


############## Aplicamos la función ##############


,Sex,DVRT,Educational_level,Prestige_score,Type_school,Leaving_certificate
0,MALE,113,Junior_cycle_incomplete-secondary_school,28.0,secondary,not_taken
1,MALE,101,Primary_terminal_leaver,28.0,primary_terminal_leaver,not_taken
2,MALE,110,Senior_cycle_terminal_leaver-secondary_school,69.0,secondary,taken
3,MALE,121,Junior_cycle_terminal_leaver-secondary_school,57.0,secondary,not_taken
4,MALE,82,Junior_cycle_terminal_leaver-vocational_school,18.0,vocational,not_taken
...,...,...,...,...,...,...
495,MALE,137,3rd_level_complete,62.0,secondary,taken
496,MALE,136,3rd_level_complete,18.0,secondary,taken
497,MALE,132,3rd_level_complete,37.0,secondary,taken
498,FEMALE,135,3rd_level_complete,62.0,secondary,taken


## 3.3 Aplicando filtros
Una forma de quedarnos con las filas que nos interesan es mediante la utilización de una máscara. Una máscara es un array de `boolean` que indica las filas seleccionadas. Si le pasamos la máscara al `DataFrame` (`df[máscara]`) entonces nos devolverá únicamente las filas para las que el array tiene almacenado `True`.

In [12]:
print('\n############## Personas con "Prestige_score" inferior a 20 ##############')
mask = df['Prestige_score'] < 20
display(df[mask])


############## Personas con "Prestige_score" inferior a 20 ##############


,Sex,DVRT,Educational_level,Prestige_score,Type_school,Leaving_certificate
4,male,82,Junior_cycle_terminal_leaver-vocational_school,18.0,vocational,not_taken
9,male,90,Primary_terminal_leaver,18.0,primary_terminal_leaver,not_taken
11,male,84,Junior_cycle_terminal_leaver-vocational_school,18.0,vocational,not_taken
12,male,114,3rd_level_complete,18.0,secondary,taken
20,male,86,Primary_terminal_leaver,18.0,primary_terminal_leaver,not_taken
...,...,...,...,...,...,...
460,female,104,Junior_cycle_terminal_leaver-secondary_school,18.0,secondary,not_taken
469,male,125,Senior_cycle_terminal_leaver-secondary_school,18.0,secondary,taken
470,male,129,Junior_cycle_terminal_leaver-secondary_school,18.0,secondary,not_taken
487,female,122,Junior_cycle_terminal_leaver-vocational_school,18.0,vocational,not_taken


Estas instrucciones se suelen escribir de forma más compacta en una sola línea:

In [13]:
print('\n############## Personas con "Prestige_score" inferior a 20 ##############')
display(df[df['Prestige_score'] < 20])


############## Personas con "Prestige_score" inferior a 20 ##############


,Sex,DVRT,Educational_level,Prestige_score,Type_school,Leaving_certificate
4,male,82,Junior_cycle_terminal_leaver-vocational_school,18.0,vocational,not_taken
9,male,90,Primary_terminal_leaver,18.0,primary_terminal_leaver,not_taken
11,male,84,Junior_cycle_terminal_leaver-vocational_school,18.0,vocational,not_taken
12,male,114,3rd_level_complete,18.0,secondary,taken
20,male,86,Primary_terminal_leaver,18.0,primary_terminal_leaver,not_taken
...,...,...,...,...,...,...
460,female,104,Junior_cycle_terminal_leaver-secondary_school,18.0,secondary,not_taken
469,male,125,Senior_cycle_terminal_leaver-secondary_school,18.0,secondary,taken
470,male,129,Junior_cycle_terminal_leaver-secondary_school,18.0,secondary,not_taken
487,female,122,Junior_cycle_terminal_leaver-vocational_school,18.0,vocational,not_taken


Podemos aplicar varias condiciones para que se vayan filtrando las filas y así quedarnos con las que nos interesan. Para ello podemos combinar los operadores *element-wise* para la conjunción (`&`), disyunción (`|`) y negación (`~`).

Veamos un ejemplo donde se aplica la conjunción:

In [14]:
print('\n############## Personas con "Prestige_score" inferior a 20 y escuela "vocational" ##############')
display(df[(df['Prestige_score'] < 20) & (df['Type_school'] == 'vocational')])


############## Personas con "Prestige_score" inferior a 20 y escuela "vocational" ##############


,Sex,DVRT,Educational_level,Prestige_score,Type_school,Leaving_certificate
4,male,82,Junior_cycle_terminal_leaver-vocational_school,18.0,vocational,not_taken
11,male,84,Junior_cycle_terminal_leaver-vocational_school,18.0,vocational,not_taken
61,male,99,Junior_cycle_terminal_leaver-vocational_school,18.0,vocational,not_taken
130,female,70,Junior_cycle_incomplete-vocational_school,18.0,vocational,not_taken
165,male,86,Junior_cycle_incomplete-vocational_school,18.0,vocational,not_taken
184,female,105,Senior_cycle_terminal_leaver-secondary_school,18.0,vocational,taken
185,female,70,Junior_cycle_incomplete-vocational_school,18.0,vocational,not_taken
190,female,87,Senior_cycle_terminal_leaver-secondary_school,18.0,vocational,taken
210,female,91,Junior_cycle_incomplete-vocational_school,18.0,vocational,not_taken
214,female,100,Junior_cycle_terminal_leaver-vocational_school,18.0,vocational,not_taken


Y ahora un último ejemplo con la negación:

In [15]:
print('\n############## Personas con "Prestige_score" inferior a 20, escuela "vocational" y con certificado "no not_taken" ##############')
display(df[(df['Prestige_score'] < 20) & (df['Type_school'] == 'vocational') & ~(df['Leaving_certificate'] == 'not_taken')])


############## Personas con "Prestige_score" inferior a 20, escuela "vocational" y con certificado "no not_taken" ##############


,Sex,DVRT,Educational_level,Prestige_score,Type_school,Leaving_certificate
184,female,105,Senior_cycle_terminal_leaver-secondary_school,18.0,vocational,taken
190,female,87,Senior_cycle_terminal_leaver-secondary_school,18.0,vocational,taken
273,male,118,Senior_cycle_terminal_leaver-secondary_school,18.0,vocational,taken


## 3.4 Concatenación de `DataFrames`
Si queremos concatenar dos `DataFrames` podemos utilizar la función `concat()`.

En el siguiente ejemplo se crean dos `DataFrames` pequeños, el primero con las filas 7 y 8 de `df` y el segundo con las filas 5, 6 y 7 para, posteriormente, concatenarlos.

In [16]:
print('\n############## se crea df1 con las filas 7 y 8 de df ##############')
df1 = df.loc[7:8]
display(df1)

print('\n############## se crea df2 con las filas 5, 6 y 7 de df ##############')
df2 = df.loc[5:7]
display(df2)

print('\n############## se concatenan df1 y df2 ##############')
display(pd.concat([df1, df2], axis=0))


############## se crea df1 con las filas 7 y 8 de df ##############


,Sex,DVRT,Educational_level,Prestige_score,Type_school,Leaving_certificate
7,male,98,Junior_cycle_incomplete-vocational_school,43.0,vocational,not_taken
8,male,92,Junior_cycle_terminal_leaver-vocational_school,33.0,vocational,not_taken



############## se crea df2 con las filas 5, 6 y 7 de df ##############


,Sex,DVRT,Educational_level,Prestige_score,Type_school,Leaving_certificate
5,male,85,Junior_cycle_terminal_leaver-vocational_school,28.0,vocational,not_taken
6,male,84,Primary_terminal_leaver,NaN,primary_terminal_leaver,not_taken
7,male,98,Junior_cycle_incomplete-vocational_school,43.0,vocational,not_taken



############## se concatenan df1 y df2 ##############


,Sex,DVRT,Educational_level,Prestige_score,Type_school,Leaving_certificate
7,male,98,Junior_cycle_incomplete-vocational_school,43.0,vocational,not_taken
8,male,92,Junior_cycle_terminal_leaver-vocational_school,33.0,vocational,not_taken
5,male,85,Junior_cycle_terminal_leaver-vocational_school,28.0,vocational,not_taken
6,male,84,Primary_terminal_leaver,NaN,primary_terminal_leaver,not_taken
7,male,98,Junior_cycle_incomplete-vocational_school,43.0,vocational,not_taken


Al poner el parámetro `axis` a 0 se concatenan verticalmente. Si lo hubiésemos puesto a 1 se concatenarían horizontalmente.

Algo que nos puede llamar la atención es que aparecen los índices de las filas que originalmente tenían en `df` y, por tanto, en el ejemplo aparecen dos filas con el número 7. Cuando hagamos una catenación, si esos índices originales no son relevantes, lo mejor es utilizar el parámetro `ignore_index=True`, lo que hará que se reseteen los índices.

In [18]:
print('\n############## se concatenan df1 y df2 y se resetean los índices ##############')
display(pd.concat([df1, df2], axis=0, ignore_index=True))



############## se concatenan df1 y df2 y se resetean los índices ##############


,Sex,DVRT,Educational_level,Prestige_score,Type_school,Leaving_certificate
0,male,98,Junior_cycle_incomplete-vocational_school,43.0,vocational,not_taken
1,male,92,Junior_cycle_terminal_leaver-vocational_school,33.0,vocational,not_taken
2,male,85,Junior_cycle_terminal_leaver-vocational_school,28.0,vocational,not_taken
3,male,84,Primary_terminal_leaver,NaN,primary_terminal_leaver,not_taken
4,male,98,Junior_cycle_incomplete-vocational_school,43.0,vocational,not_taken


## Ejercicios

Haz un programa que cargue el fichero **biomed.data** (es un archivo de texto) y realice lo siguiente:
1. Agrupa por paciente ('Observation_number') y calcula los valores medios
2. Agrupa por paciente y clase y calcula la media de 'm1'
3. Agrupa por paciente y clase y calcula la media y desviación de la edad y el máximo y el mínimo de m4
4. Cambia la edad a meses y el nombre de la columna a 'Months' (busca ayuda de la función `rename()` para renombrar una columna)
5. Haz un filtro para ver las filas en las que 'm3' sea menor que 10 o mayor que 100 y que la clase NO sea normal

Estos ejercicios no es necesario entregarlos.